In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing


2023-01-04 13:32:25.736216: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-04 13:32:25.843386: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-04 13:32:25.847342: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-04 13:32:25.847355: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [3]:
latent_dim = 3
inputlen = 27
units = 514

# Encoder
encoder_inputs = keras.Input(shape=(inputlen,))
x = layers.Dense(units=units, activation='relu')(encoder_inputs)
x = layers.Dense(units=units//2, activation='relu')(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

# Decoder
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(units=units//2, activation='relu')(latent_inputs)
x = layers.Dense(units=units, activation='relu')(x)
decoder_outputs = layers.Dense(inputlen, activation='linear')(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 27)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 514)          14392       ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 257)          132355      ['dense[0][0]']                  
                                                                                                  
 z_mean (Dense)                 (None, 3)            774         ['dense_1[0][0]']                
                                                                                            

2023-01-04 13:32:27.517693: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-04 13:32:27.517712: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-04 13:32:27.517726: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vmhpc): /proc/driver/nvidia/version does not exist
2023-01-04 13:32:27.517906: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #print("reconstruction = " + str(reconstruction))
            #print("z_mean = " + str(z_mean)+"\n")
            #print("z_log_var = " + str(z_log_var)+"\n")
            #print("z = " + str(z)+"\n")
            
            reconstruction_loss = keras.losses.mean_squared_error(data, reconstruction)
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            #print("First kl_loss = "+str(kl_loss)+"\n")
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss,axis=1)) 
            #mprint("Second kl_loss = "+str(kl_loss)+"\n")
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [5]:
df = pd.read_csv("/home/vmh/vmhdocs/Research/Inria/Anl/EnvDataAutoencoder/Data/Imputed_Metadata_metaGmetaT_by_OceanLayer.csv",sep=",")
selectcols = ["Latitude","Longitude","Depth.nominal","Temperature","Oxygen","ChlorophyllA","Carbon.total","Salinity","Gradient.Surface.temp","Fluorescence","CO3","HCO3","Density","PO4","NO3","Si","Alkalinity.total","Ammonium.5m","Lyapunov","NO2","Depth.Min.O2","NO2NO3","Nitracline","Brunt.Vaisala","Iron.5m","Depth.Max.O2","Okubo.Weiss"]
df = df[selectcols]
print(df.shape)
df = preprocessing.RobustScaler().fit_transform(df)



(228, 27)


In [6]:
vae = VAE(encoder, decoder)
vae.load_weights('./vae_checkpoint/my_checkpoint')

In [7]:
n = 5
scale = 1.0
grid_x = np.linspace(-scale, scale, n)
grid_y = np.linspace(-scale, scale, n)
grid_z = np.linspace(-scale, scale, n)

for i, xi in enumerate(grid_x):
    for j, yi in enumerate(grid_y):
        for k, zi in enumerate(grid_z): 
            z_sample = np.array([[xi, yi, zi]])
            decoded = vae.decoder.predict(z_sample)
            print(z_sample)
            print(decoded)
            print(decoded.shape)
            print("")

1/1 [==============================] - 0s 79ms/step
[[-1. -1. -1.]]
[[ 0.9824776   0.40117875  0.17846557 -0.61401194  2.2490368   3.7239614
   6.666686   -0.96338046  0.8756459   0.6230613   0.48282614  1.6144099
  -0.01584408  0.02683302  0.0766172   0.59939593  1.7256424   0.17703596
  -0.25317007  0.29956794 -0.04462655  0.10198986  0.86034    -0.12790796
  -0.20590556  0.23610805 -0.01752474]]
(1, 27)

1/1 [==============================] - 0s 12ms/step
[[-1.  -1.  -0.5]]
[[ 0.09826872  0.18547909  0.4577613   0.06103278  0.4652141   0.7153983
   0.750572    0.19512887  0.28244275 -0.18017271  0.45586914  1.1083237
  -0.24308106  0.06634831  0.16018872  0.07282072  0.52301335  0.6624736
   0.1746993   0.45456383  0.04027347  0.21981764  0.45452395  0.33818436
  -0.11419614  0.72905356  0.50352484]]
(1, 27)

1/1 [==============================] - 0s 26ms/step
[[-1. -1.  0.]]
[[ 0.0060313   0.160772    0.622335    0.08497655  0.29235414  0.5069927
   0.3112252   0.25845644  0.300689